read and process the video file

In [2]:
#! /usr/bin/python
import numpy as np
import cv2
cv2.destroyAllWindows()
cap = cv2.VideoCapture('data/IMGP1653.MOV') # insect eye
# cap = cv2.VideoCapture('data/IMGP1421.MOV') # denim
# cap = cv2.VideoCapture('data/IMGP1652.MOV') # insect face
# cap = cv2.VideoCapture('data/IMGP1661.MOV') # coin
# cap = cv2.VideoCapture('data/IMGP1659.MOV') # scrath on paper
# cap = cv2.VideoCapture('data/IMGP1654.MOV') # insect leg
stop_frame = 800
gradients = []
frames = []
iteration = 0
pause = False
break_loop = False

while not break_loop:
    if not pause:
        ret, frame = cap.read()
        if not ret:
            break
        if iteration == stop_frame:
            break
        iteration += 1
        # print(iteration)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.blur(gray, (3,3)) 
        laplacian = cv2.Laplacian(gray,cv2.CV_64F, ksize=3)
        # laplacian = cv2.Sobel(src=gray, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=3) # Combined X and Y Sobel Edge Detection
        # laplacian = cv2.Laplacian(gray,cv2.CV_64F)

        laplacian = cv2.convertScaleAbs( laplacian )

        gradients.append(laplacian)
        frames.append(frame)

        effective_image = np.vstack((laplacian, gray))

        width = int(effective_image.shape[1] * 0.4)
        height = int(effective_image.shape[0] * 0.4)
        dim = (width, height)
        cv2.imshow('window', cv2.resize(effective_image, dim, interpolation = cv2.INTER_AREA))
    
    k = cv2.waitKey(1)

    if k == ord('p'):
        pause = not pause
        print("pause", pause)
    if k == ord('q'):
        break_loop = True 



Qt: Session management error: Could not open network socket


get the depth.

In [3]:
cv2.destroyAllWindows()
gradients = np.array(gradients)
maxima = np.amax(gradients, axis=0)
# maxima = cv2.blur(maxima, (3,3))

width = int(maxima.shape[1] * 0.4)
height = int(maxima.shape[0] * 0.4)
dim = (width, height)
indexes = np.array(gradients.argmax(axis=0)).astype(np.uint32)
indexes_blurred = cv2.blur(indexes.astype(np.uint8), (13,13))
mxm = cv2.resize(maxima, dim, interpolation = cv2.INTER_AREA).astype(np.float)
cv2.imshow('gradient_maxima', mxm.astype(np.uint8))
mxm*=255.0/np.amax(mxm)
cv2.imshow('gradient_maxima_jet', cv2.applyColorMap(mxm.astype(np.uint8), cv2.COLORMAP_JET))
cv2.imshow('gradient_maxima_indices', cv2.resize(indexes.astype(np.uint8), dim, interpolation = cv2.INTER_AREA))

k = cv2.waitKey(10)
threshold =  np.percentile(maxima, 10)
# threshold =  150
print("threshold: ",threshold)
print('saving')
width = int(maxima.shape[1])
height = int(maxima.shape[0])
focused_wb = np.zeros((height, width, 3), np.uint8)
focused = np.zeros((height, width, 3), np.uint8)

# scatter_points = []

# scatter_np_points = np.array[]
# scatter_np_points = 


threshold:  8.0
saving


the following section displays the result using open3d library

In [4]:

cv2.destroyAllWindows()
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d


(r, c) = np.shape(indexes)
# print (r,c,b)
(X, Y) = np.meshgrid(range(0, c), range(0, r))

scatter_points = np.zeros((1080,1920,3))
scatter_points[:,:,0] = X
scatter_points[:,:,1] = Y
scatter_points[:,:,2] = indexes*5
shp = scatter_points.shape
scatter_points = scatter_points.transpose(1,0,2).reshape(shp[1],-1,shp[-1])
scatter_points = scatter_points.reshape(1920*1080,3)
pcd = o3d.geometry.PointCloud()

# print("slicing color")
# color = np.choose(frames.reshape(1920*1080,3), indexes)
# print(color)
pcd.points = o3d.utility.Vector3dVector(scatter_points)
# colors = np.array(scatter_points)[:,3:6]
pcd.colors = o3d.utility.Vector3dVector(colors/256.0)
# cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
cl, ind = pcd.remove_radius_outlier(nb_points=16, radius=10)
# cl = cl.voxel_down_sample(voxel_size=10)
o3d.visualization.draw_geometries([cl])
# o3d.visualization.draw_geometries([pcd])
cv2.destroyAllWindows()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
slicing color


ValueError: Need at least 0 and at most 32 array objects.

In [ ]:

with open("pc.obj", 'w') as f:
    for x in range(width):
        for y in range(height):
            factor = 5
            # if maxima[y,x] > 220:

            color = frames[int(indexes[y,x])][y,x]
            # grad = [maxima[y,x]] * 20
            focused_wb[y,x,:] = color
            if maxima[y,x] > threshold:
                focused[y,x,:] = color
                point = list([x,y,indexes[y,x]*factor]) + list(reversed(color))
                scatter_points.append(point)
                # print(color)
                # print(color)
                # print(*list(color))
                f.write("v %f %f %f %d %d %d\n" % (*list(point),)) # do not write eventually
            else:
                indexes_blurred[y,x] = 0        
        cv2.imshow('filtered', cv2.resize(indexes_blurred, dim, interpolation = cv2.INTER_AREA))
        k = cv2.waitKey(1)
print('done')

[[ 0  2 75]]
[[  0   2  75]
 [  0   3 245]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]
 [  0   6  55]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]
 [  0   6  55]
 [  0   7  55]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]
 [  0   6  55]
 [  0   7  55]
 [  0   9 315]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]
 [  0   6  55]
 [  0   7  55]
 [  0   9 315]
 [  0  10 400]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]
 [  0   6  55]
 [  0   7  55]
 [  0   9 315]
 [  0  10 400]
 [  0  11 305]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]
 [  0   6  55]
 [  0   7  55]
 [  0   9 315]
 [  0  10 400]
 [  0  11 305]
 [  0  13  40]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]
 [  0   6  55]
 [  0   7  55]
 [  0   9 315]
 [  0  10 400]
 [  0  11 305]
 [  0  13  40]
 [  0  14  55]]
[[  0   2  75]
 [  0   3 245]
 [  0   5 405]
 [  0   6  55]
 [  0   7  55]
 [  0   9 315]
 [  0  10 400]
 [  0  11 305]
 [  0  13  40]
 [  0  14  55]
 [  0  16 110]]
[[

KeyboardInterrupt: 